<a href="https://colab.research.google.com/github/C-T-W/water-quality-nyc/blob/main/water_quality_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install sodapy
! pip install geopandas 

# Import modules
# --------------------
import re                                         # For regular expressions
import numpy as np 
import pandas as pd
import geopandas                                  # For geometric and coordinate data
import matplotlib.pyplot as plt                   # For plotting data              
from sodapy import Socrata                        # For importing 
from shapely.geometry import Point,MultiPoint
from shapely.ops import nearest_points

     |████████████████████████████████| 972kB 5.5MB/s 
     |████████████████████████████████| 14.8MB 315kB/s 
     |████████████████████████████████| 6.5MB 50.1MB/s 


In [8]:
# Upload data files into local directory
! curl 'https://raw.githubusercontent.com/C-T-W/water-quality-nyc/main/data/Distribution_Water_Quality_Sampling_Sites_for_OpenData.xlsx' -o sample_sites.xlsx
! curl 'https://raw.githubusercontent.com/C-T-W/water-quality-nyc/main/data/us-zip-code-latitude-and-longitude.csv' -o us_zip_code_latitude_and_longitude.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32750  100 32750    0     0   180k      0 --:--:-- --:--:-- --:--:--  180k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146k  100  146k    0     0  1404k      0 --:--:-- --:--:-- --:--:-- 1404k


In [10]:
# Create a 'coordinates_df' data frame to store the x and y coordinates given for each sampling site within the data set
coordinates_df = pd.read_excel('sample_sites.xlsx')

# We drop 'Location Description' column to update df to only contain 'Site', which we will use as key, along with X and Y Coordinate columns as values
coordinates_df = coordinates_df.drop(columns='Sample Station (SS) - LOCATION DESCRIPTION')

# Coordinate Reference Systems
# ---------------------------------
# The x-y coordinates of each sample site were given, based on a specific “geographic coordinate system” (GCS),
# and a more specific “projected coordinate system” (PCS).

# The spatial reference ID for the PCS used for data set coordinates in geopandas library
# SRID = "ESRI:102718"
# Now convert coordinates_df into GeoData frame
# ------------------------------------------------
# Sample Site (SS) GeoDataFrame needs a 'shapely' object
# We create this object by turning the coordinates provided into 'Point' objects, and we specify the coordinate reference system (CRS) using the spatial reference ID provided for our PCS
ss_gdf = geopandas.GeoDataFrame(coordinates_df, geometry=geopandas.points_from_xy(coordinates_df['X - Coordinate'],coordinates_df['Y - Coordinate']), crs="ESRI:102718" )

ss_gdf.crs # Check the coordinate reference system (CRS) of the GeoDataframe, will be "ESRI:102718". This is what the x-y coordinates are referenced by

ss_gdf = ss_gdf.to_crs("EPSG:4326") # Change CRS to WGS84 Latitude/Longitude by using SRID 'EPSG:4326", converting all of the x-y coordinates to lat/long

ss_gdf.dtypes


Site                object
X - Coordinate       int64
Y - Coordinate       int64
geometry          geometry
dtype: object

In [11]:
# We now have each sample site location with a latitude/longitude point
# Next step is to create latitude/longitude point objects for zip code information
# With this data we'll be able to find nearest zipcode coordinates for each sample site coordinate
# -----
# Read in zipcode file into 'zipcode_df'
# CSV file uses semicolon as separator so pass in ';' for sep parameter
zipcode_df = pd.read_csv('us_zip_code_latitude_and_longitude.csv', sep=';')

zipcode_df[['City','Zip']] = zipcode_df[['City','Zip']].astype(str)

# Create GeoDataFrame from zipcode_df with latitude and longitude column values as x and y for point object - Specify CRS as WGS84 Latitude/Longitude
zip_gdf = geopandas.GeoDataFrame(zipcode_df, geometry=geopandas.points_from_xy(zipcode_df['Longitude'],zipcode_df['Latitude']), crs='EPSG:4326')

# We are only interested in the newly created geometry column containing lat/long point, zip column, and the city column
# Drop all other columns and save to original Geodataframe
zip_gdf = zip_gdf.drop(columns=['State','Latitude','Longitude','Timezone','Daylight savings time flag','geopoint'])

zip_gdf.dtypes

Zip           object
City          object
geometry    geometry
dtype: object

In [12]:
# Now that we have both zipcode and sample site location data as latitude/longitude points we want to check the distances between them
# Based on which zipcode location the site location is closest to, we'll add that zipcode and borough to the samples df for that specific sample site

# First put all of the zip code points into a single multipoint object
# This will be used to iterate over as we check the distances between these points and each sample site
zipcode_pts = MultiPoint(zip_gdf['geometry'])

# Function takes in geo point and returns a series containing the city and zipcode values closest to the point parameter
def nearest_zipcity(pt):
  nearest_pts = nearest_points(pt,zipcode_pts)                          # Create tuple containing original sample site point and closest zip code point using 'nearest_points' function
  zip_pt = nearest_pts[1]                                               # Assign 'zip_pt' value of coordinates for zipcode point, which is the second value of tuple returned by previous function
  city = zip_gdf.loc[zip_gdf['geometry'] == zip_pt, 'City'].values[0]   # Locate the city associated with the zip code coordinates given
  zipcode = zip_gdf.loc[zip_gdf['geometry'] == zip_pt, 'Zip'].values[0] # Locate the zipcode for coordinates given
  return pd.Series({'City': city, 'Zipcode': zipcode})                  # Return a series containing the city and zipcode obtained for the given sampling site point

ss_gdf[['City','Zipcode']] = ss_gdf['geometry'].apply(nearest_zipcity)  # Create new columns in sampling site gdf with city and zipcode info returned from function

ss_gdf = ss_gdf.drop(columns=['X - Coordinate','Y - Coordinate'])       # Drop columns with original projected coordinate system info


In [13]:
# Sample site GeoDataframe containing point (longitude,latitude), City, and Zipcode for each Sample Site identifier
ss_gdf

,Site,geometry,City,Zipcode
0,1S03,POINT (-73.85279 40.89196),Bronx,10466
1,1S04,POINT (-73.89761 40.87087),Bronx,10468
2,1S03A,POINT (-73.85362 40.89228),Bronx,10466
3,1S07,POINT (-73.92858 40.83976),Bronx,10452
4,1S03B,POINT (-73.88898 40.88215),Bronx,10458
...,...,...,...,...
394,78850,POINT (-73.79146 40.69338),Jamaica,11433
395,79050,POINT (-73.75669 40.66542),Springfield Gardens,11413
396,79150,POINT (-73.76115 40.68085),Jamaica,11434
397,79450,POINT (-73.80984 40.67589),South Ozone Park,11420


In [14]:
# Locate the sample site(s) that reside within the zipcode of my home
home_zip_ss = ss_gdf.loc[ss_gdf['Zipcode'] == '10463'] # Check for 'Zipcode' column value

home_zip_ss

,Site,geometry,City,Zipcode
46,15300,POINT (-73.90235 40.87824),Bronx,10463
56,17350,POINT (-73.91225 40.88399),Bronx,10463
61,17950,POINT (-73.90529 40.88123),Bronx,10463


In [15]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Import Water Quality Data set
results = client.get('bkwf-xfky', limit=85300)

# Convert dataset to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [16]:
results_df

# Rename column 'sample_site' to 'Site' to match sample site geodataframe for merging
results_df = results_df.rename(columns={'sample_site': 'Site'})

In [17]:
results_df.shape

(85300, 11)

In [18]:
# Merge results df and sample sites data frame on 'Site', 'City', and 'Zipcode' columns and assign to NYC_samples_df
NYC_samples_df = pd.merge(results_df, ss_gdf[['Site','City','Zipcode']], how='left')


In [19]:
# Rename columns to final 'City' and 'Zipcode' names
NYC_samples_df = NYC_samples_df.rename(columns={'City_x': 'City', 'Zipcode_x': 'Zipcode'})

# Drop column containing textual description of sample site location
NYC_samples_df = NYC_samples_df.drop(columns='location')


In [20]:
# Display shape of dataframe - (Rows, Columns)
NYC_samples_df.shape

(85484, 12)

In [21]:
# Dataframe column datatypes
NYC_samples_df.dtypes

sample_number                     object
sample_date                       object
sample_time                       object
Site                              object
sample_class                      object
residual_free_chlorine_mg_l       object
turbidity_ntu                     object
coliform_quanti_tray_mpn_100ml    object
e_coli_quanti_tray_mpn_100ml      object
fluoride_mg_l                     object
City                              object
Zipcode                           object
dtype: object

In [22]:
# Sample class column contains many different values but we will only want to work with the main 2 categories
# Show distribution of rows for each sample_class in original df
NYC_samples_df['sample_class'].value_counts()

Compliance              52591
Operational             32126
Resample_Compliance       474
Entry Point               168
Re-Sample                  93
Op-resample                31
Resample_Operational        1
Name: sample_class, dtype: int64

In [23]:
# Could not find information regarding the sample classes other than 'Compliance' or 'Operational'
# Only keep rows where 'sample_class' is equal to one of those two classes
NYC_samples_df = NYC_samples_df[(NYC_samples_df['sample_class'] == 'Compliance') | (NYC_samples_df['sample_class'] == 'Operational')].copy()


In [24]:
# Show the 767 rows that did not contain Compliance or Operational sample class values have been removed, columns remain unchanged
NYC_samples_df.shape

(84717, 12)

In [25]:
# Display new distribution of rows among sample_class column
NYC_samples_df['sample_class'].value_counts()

Compliance     52591
Operational    32126
Name: sample_class, dtype: int64

In [26]:
# Update dataype of 'sample_class' column to categorical to represent the 'Compliance' and 'Operational' sample classes
# Convert the categorical sample_class colummn to categorical datatype
# ------------------------
NYC_samples_df['sample_class'] = pd.Categorical(NYC_samples_df['sample_class'],categories = ['Compliance', 'Operational'])


In [27]:
# Convert the numeric measurement colummns to numeric datatypes
# ------------------------
# Create list of column names to be converted to numeric
col = ['residual_free_chlorine_mg_l','turbidity_ntu','fluoride_mg_l']

# Apply to_numeric function to all columns included in list
NYC_samples_df[col] = NYC_samples_df[col].apply(pd.to_numeric, errors='coerce')

In [28]:
NYC_samples_df.dtypes

sample_number                       object
sample_date                         object
sample_time                         object
Site                                object
sample_class                      category
residual_free_chlorine_mg_l        float64
turbidity_ntu                      float64
coliform_quanti_tray_mpn_100ml      object
e_coli_quanti_tray_mpn_100ml        object
fluoride_mg_l                      float64
City                                object
Zipcode                             object
dtype: object

In [29]:
NYC_samples_df.describe()

,residual_free_chlorine_mg_l,turbidity_ntu,fluoride_mg_l
count,84710.000000,84217.000000,10896.000000
mean,0.586467,0.725579,0.711620
std,0.212181,0.271040,0.050238
min,-9.990000,0.070000,0.030000
25%,0.450000,0.620000,0.690000
50%,0.590000,0.720000,0.710000
75%,0.730000,0.830000,0.730000
max,2.200000,33.800000,0.890000


In [30]:
# Within the 'residual_free_chlorine_mg_l' column there is a row with a value that does not make sense
# Cannot have a negative value for a concentration of chlorine in a sample
# Set this row value to 0.0 so that we can still use row and it will not impact the decriptive statistics when used
NYC_samples_df.loc[NYC_samples_df.residual_free_chlorine_mg_l == -9.99, 'residual_free_chlorine_mg_l'] = 0.0

NYC_samples_df.describe()

,residual_free_chlorine_mg_l,turbidity_ntu,fluoride_mg_l
count,84710.000000,84217.000000,10896.000000
mean,0.586585,0.725579,0.711620
std,0.209056,0.271040,0.050238
min,0.000000,0.070000,0.030000
25%,0.450000,0.620000,0.690000
50%,0.590000,0.720000,0.710000
75%,0.730000,0.830000,0.730000
max,2.200000,33.800000,0.890000


In [31]:
# Convert Date columns
# ------------------------

# Update datatype of 'sample_date' column to datetime format
NYC_samples_df['sample_date'] = pd.to_datetime(NYC_samples_df['sample_date'],format='%Y-%m-%d')

# Create new columns in the df with values for the year, month, and day of sample based on 'sample_date' column
NYC_samples_df['year'] = NYC_samples_df['sample_date'].dt.year
NYC_samples_df['month'] = NYC_samples_df['sample_date'].dt.month
NYC_samples_df['day'] = NYC_samples_df['sample_date'].dt.day


In [32]:
# Display how many records belong to each 'city' or borough in NYC
NYC_samples_df['City'].value_counts()

Bronx                  21016
Brooklyn               19025
New York               13877
Staten Island          10193
Astoria                 3034
Jamaica                 1924
Queens Village          1389
Flushing                1116
Cambria Heights          549
Hollis                   543
East Elmhurst            537
Far Rockaway             497
Little Neck              473
Bayside                  467
Rego Park                463
Jackson Heights          456
Elmhurst                 451
Fresh Meadows            422
Saint Albans             363
South Richmond Hill      363
Maspeth                  363
Springfield Gardens      362
Ozone Park               357
Howard Beach             306
Oakland Gardens          292
Whitestone               289
Sunnyside                287
Long Island City         286
Middle Village           276
Woodside                 271
South Ozone Park         188
Arverne                  187
Rockaway Park            186
Richmond Hill            186
Breezy Point  

In [33]:
# We would like to group by borough
# Remove 'Garden City' Long Island records and combine records for each neighborhood in Queens

# Drop samples along index where df 'City' is equal to 'Garden City', inplace = True so the drop is applied to original df
NYC_samples_df.drop(NYC_samples_df.index[NYC_samples_df.City == 'Garden City'], inplace=True)

# Condition for rows that are not in the four boroughs already, i.e. the neighborhoods in Queens
queens_rows = ((NYC_samples_df['City'] != 'Bronx') & 
              (NYC_samples_df['City'] != 'Brooklyn') & 
              (NYC_samples_df['City'] != 'New York') & 
              (NYC_samples_df['City'] != 'Staten Island'))

# Samples that are in queens
NYC_samples_df[queens_rows].head()

,sample_number,sample_date,sample_time,Site,sample_class,residual_free_chlorine_mg_l,turbidity_ntu,coliform_quanti_tray_mpn_100ml,e_coli_quanti_tray_mpn_100ml,fluoride_mg_l,City,Zipcode,year,month,day
5,34890,2020-11-30,11:04,40200,Operational,1.01,0.88,<1,<1,NaN,Astoria,11105,2020,11,30
6,34891,2020-11-30,11:40,44350,Compliance,0.31,0.33,<1,<1,NaN,Astoria,11106,2020,11,30
15,34910,2020-11-30,9:07,43250,Compliance,0.79,0.71,<1,<1,NaN,Woodhaven,11421,2020,11,30
17,34911,2020-11-30,10:32,45250,Compliance,0.66,0.78,<1,<1,NaN,Arverne,11692,2020,11,30
18,34912,2020-11-30,8:23,46850,Compliance,0.79,0.70,<1,<1,NaN,Maspeth,11378,2020,11,30


In [34]:
# Set value of 'City' to 'Queens' for all rows that meet 'queens_row' condition in df
NYC_samples_df.loc[((NYC_samples_df['City'] != 'Bronx') & 
              (NYC_samples_df['City'] != 'Brooklyn') & 
              (NYC_samples_df['City'] != 'New York') & 
              (NYC_samples_df['City'] != 'Staten Island')), 'City'] = 'Queens'

# Display new value counts for City column
NYC_samples_df.City.value_counts()

Bronx            21016
Queens           20511
Brooklyn         19025
New York         13877
Staten Island    10193
Name: City, dtype: int64

In [35]:
# Now that we have 5 borough values, make 'City' column into categorical datatype with values for each borough
# Convert the categorical City colummn to categorical datatype
# ------------------------
# Categories are each borough: Bronx, Queens, Brooklyn, New York, Staten Island
NYC_samples_df['City'] = pd.Categorical(NYC_samples_df['City'],categories = ['Bronx', 'Queens', 'Brooklyn', 'New York', 'Staten Island'])

In [36]:
# Display datatypes after conversion of columns to appropriate datatypes
NYC_samples_df.dtypes

sample_number                             object
sample_date                       datetime64[ns]
sample_time                               object
Site                                      object
sample_class                            category
residual_free_chlorine_mg_l              float64
turbidity_ntu                            float64
coliform_quanti_tray_mpn_100ml            object
e_coli_quanti_tray_mpn_100ml              object
fluoride_mg_l                            float64
City                                    category
Zipcode                                   object
year                                       int64
month                                      int64
day                                        int64
dtype: object

In [63]:
# Display number of records for each borough in the dataframe for every year included
NYC_samples_df.pivot_table( index='City',
                            aggfunc=np.size,
                            columns='year',
                            margins= True
                           )

Site                    ... turbidity_ntu                        
year           2015  2016  2017  2018  ...          2018    2019    2020     All
City                                   ...                                      
Bronx          3078  3130  4047  3720  ...        3720.0  3890.0  3151.0  8739.0
Queens         3160  3214  3779  3566  ...        3566.0  3871.0  2921.0   238.0
Brooklyn       2983  2957  3556  3259  ...        3259.0  3530.0  2740.0   177.0
New York       2174  2184  2628  2388  ...        2388.0  2562.0  1941.0   441.0
Staten Island  1570  1587  1907  1717  ...        1717.0  1880.0  1532.0   187.0
All             849  1641  2017  1827  ...        1827.0  1852.0  1596.0  9782.0

[6 rows x 91 columns]

In [64]:
NYC_samples_df.pivot_table( index='City',
                            aggfunc='size',
                            columns='year',
                           )

year,2015,2016,2017,2018,2019,2020
City,,,,,,
Bronx,3078,3130,4047,3720,3890,3151
Queens,3160,3214,3779,3566,3871,2921
Brooklyn,2983,2957,3556,3259,3530,2740
New York,2174,2184,2628,2388,2562,1941
Staten Island,1570,1587,1907,1717,1880,1532


In [65]:
print('Figure 1.a')

# Pivot table to display mean of numeric water quality measures for each year between sample classes
pd.pivot_table(
    data = NYC_samples_df,
    index = ['sample_class','year'],
    aggfunc = 'mean',
    values = ['residual_free_chlorine_mg_l','turbidity_ntu','fluoride_mg_l'],
)

Figure 1.a


fluoride_mg_l  residual_free_chlorine_mg_l  turbidity_ntu
sample_class year                                                           
Compliance   2015       0.737969                     0.535268       0.789183
             2016       0.686598                     0.520112       0.711266
             2017       0.686838                     0.532767       0.722329
             2018       0.733704                     0.514058       0.739476
             2019       0.710944                     0.505579       0.678403
             2020       0.713938                     0.502125       0.710526
Operational  2015       0.732782                     0.717744       0.771731
             2016       0.690571                     0.701584       0.695152
             2017       0.686921                     0.698296       0.738573
             2018       0.734510                     0.698802       0.746748
             2019       0.711852                     0.677196       0.685478
             2020       0.716386                     0.705318       0.737148

In [66]:
print('Figure 1.b')

# Pivot table showing mean of numeric water quality measures for each year broken down by borough
pd.pivot_table(
    data = NYC_samples_df,
    index = ['City','year'],
    aggfunc = 'mean',
    values = ['residual_free_chlorine_mg_l','turbidity_ntu','fluoride_mg_l'],
)

Figure 1.b


fluoride_mg_l  residual_free_chlorine_mg_l  turbidity_ntu
City          year                                                           
Bronx         2015       0.732763                     0.752128       0.762849
              2016       0.690309                     0.723588       0.665576
              2017       0.686842                     0.721562       0.733237
              2018       0.734411                     0.726922       0.736298
              2019       0.711902                     0.699416       0.679716
              2020       0.716608                     0.704878       0.734037
Queens        2015       0.738000                     0.598472       0.786130
              2016       0.696471                     0.571982       0.716430
              2017       0.690769                     0.555880       0.730422
              2018       0.735424                     0.565502       0.746849
              2019       0.710312                     0.576724       0.689401
              2020       0.710625                     0.540274       0.714962
Brooklyn      2015       0.730667                     0.556155       0.818843
              2016       0.690690                     0.529080       0.755675
              2017       0.687500                     0.574376       0.735621
              2018       0.726250                     0.562458       0.772090
              2019       0.712500                     0.542720       0.714805
              2020       0.714000                     0.537368       0.732266
New York      2015       0.741507                     0.480005       0.734039
              2016       0.684189                     0.461502       0.640825
              2017       0.686364                     0.451355       0.702952
              2018       0.734625                     0.439443       0.684881
              2019       0.709326                     0.398009       0.603610
              2020       0.709863                     0.422365       0.692045
Staten Island 2015       0.739310                     0.589134       0.812503
              2016       0.684063                     0.650246       0.752741
              2017       0.684324                     0.660839       0.737876
              2018       0.740294                     0.574825       0.767449
              2019       0.711944                     0.584601       0.708707
              2020       0.717500                     0.600888       0.710333

In [67]:
print('Figure 1.c')

# Pivot table to display min and max values of numeric water quality measures for each zipcode over each year 
zipcode_pivot = NYC_samples_df.pivot_table(index=['Zipcode', 'year'], values=['residual_free_chlorine_mg_l','turbidity_ntu','fluoride_mg_l'], aggfunc=['min','max'], fill_value=0)

# Specify the zipcode of my home to display
zipcode_pivot.loc[(['10463'])]

Figure 1.c


min  ...           max
             fluoride_mg_l  ... turbidity_ntu
Zipcode year                ...              
10463   2015           0.0  ...          1.32
        2016           0.0  ...          1.10
        2017           0.0  ...          1.22
        2018           0.0  ...          1.26
        2019           0.0  ...          1.04
        2020           0.0  ...          0.88

[6 rows x 6 columns]

I found that the numerical values did not differ much among each borough over the years in which samples were taken. This is explained by the purification steps taken prior to reaching the sampling sites in each borough. 

Figure 1.a showed that the only drastic difference was the turbidity level of 'Compliance' samples vs. 'Operational' samples. The compliance samples had significantly lower levels of turbidity over the course of the 5 years.

Figure 1.b displayed pretty similar results for each borough, even with the sample size being very different for each. 

Figure 1.c depicted the spread of each numerical measure for the sample sites within my home zipcode. Flouride measures were not reported.

The water quality near my home was representative of the overall quality in the city.

Bibliography:

https://www.esri.com/arcgis-blog/products/product/mapping/about-geographic-transformations-and-how-to-choose-the-right-one/

https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/analyze/?refine.state=NY&dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6InVzLXppcC1jb2RlLWxhdGl0dWRlLWFuZC1sb25naXR1ZGUiLCJvcHRpb25zIjp7InJlZmluZS5zdGF0ZSI6Ik5ZIn19LCJjaGFydHMiOlt7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQVZHIiwieUF4aXMiOiJsYXRpdHVkZSIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNGRjUxNUEifV0sInhBeGlzIjoic3RhdGUiLCJtYXhwb2ludHMiOjUwLCJzb3J0IjoiIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D

https://geopandas.org/projections.html

https://automating-gis-processes.github.io/2017/lessons/L3/nearest-neighbour.html

